# Momsitter User Sequence Analysis
Description

In [1]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import random
from quandl import get

# Data load




## Data munging
Description

마이닝에 용이하도록 데이터를 변환


## Mining

## Analysis